# Hyperparameter Tuning

This project aims to compare the performance of various regression models and hyperparameter optimization techniques using cross-validation.

Five different models are used:
  - Decision Tree Regression
  - Random Forest Regression
  - Extra Trees Regression
  - K Nearest Neighbours Regression
  - Hist Gradient Boosting Regression.

For hyperparameter optimization, four different techniques are used:
  - Grid Search
  - Random Search
  - Optuna 
  - Scikit-Optimize
  
At the end performance of these models is compared to a naive model that always predicts the mean of the target variable. The performance is evaluated based on the negative root mean squared error (RMSE) using 3-fold cross-validation. Additionally, the execution time for each model is recorded to evaluate their efficiency. Overall, this project aims to provide insight into the effectiveness of different machine learning models and optimization techniques.

In [ ]:
# Import necessary libraries

import pandas as pd
import numpy as np

from time import time

from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting, enable_halving_search_cv
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import HalvingGridSearchCV, HalvingRandomSearchCV

from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical

import optuna

In [ ]:
# Load data from CSV file
data = pd.read_csv("kaggleCompetition.csv")

# Convert data to a numpy array for faster computations
data = data.values

# Split the data into train and test sets
X = data[0:1460, :-1]
y = data[0:1460, -1]
X_comp = data[1460:, :-1]
y_comp = data[1460:, -1]

In [ ]:
np.random.seed(100469317)  # Set random seed for reproducibility

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

results = {}  # Placeholder for model results
exec_times = {}  # Placeholder for model execution times

In [ ]:
"""
if dev is True, the X_train and y_train are replaced by small random samples
to accelerate the execution while developing
"""
dev = False

if dev:
    # If in development mode, use a small random sample
    dev_sample = np.random.choice(X.shape[0], 10)
    X_train = X[dev_sample]
    y_train = y[dev_sample]

#### 1. Model Evaluation with Cross-Validation and Default Parameters
In this section of the code, the performance of five different regression models is evaluated using KFold cross-validation. The models being evaluated are Decision Tree Regression, K Nearest Neighbours Regression, Random Forest Regression, Extra Trees Regression, and Hist Gradient Boosting Regression. For each model, an instance of the model is created with default parameters and then the cross-validated negative root mean squared error is computed using the KFold cross-validator with 3 splits. The results of each model are recorded, which maps the name of the model to its performance. The execution time for each model is also recorded.

In [ ]:
# Creating an instance of KFold cross-validator with 3 splits
cv = KFold(n_splits=3)

In [ ]:
# DECISION TREE

start_time = time()

# Create a decision tree regressor with default parameters
tree_regressor_default = DecisionTreeRegressor()

# Compute the cross-validated negative root mean squared error
results['Decision tree regressor'] = np.average(
                                        cross_val_score(
                                            tree_regressor_default,
                                            X_train,
                                            y_train,
                                            cv=cv,
                                            scoring='neg_root_mean_squared_error'
                                            )
                                        )

# Record the execution time
exec_times['Decision tree regressor'] = time() - start_time

In [ ]:
# K NEAREST NEIGHBOURS

start_time = time()

# Create a k neighbours regressor with default parameters
k_neighbors_default = KNeighborsRegressor()

# Compute the cross-validated negative root mean squared error
results['KNN'] = np.average(
                    cross_val_score(
                        k_neighbors_default,
                        X_train,
                        y_train,
                        cv=cv,
                        scoring='neg_root_mean_squared_error'
                        )
                    )

# Record the execution time
exec_times['KNN'] = time() - start_time

In [ ]:
# RANDOM FOREST

start_time = time()

# Create a random forest regressor with default parameters
random_forest_default = RandomForestRegressor()

# Compute the cross-validated negative root mean squared error
results['Random forest'] = np.average(
                                cross_val_score(
                                    random_forest_default,
                                    X_train,
                                    y_train,
                                    cv=cv,
                                    scoring='neg_root_mean_squared_error'
                                    )
                                )

# Record the execution time
exec_times['Random forest'] = time() - start_time

In [ ]:
# EXTRA TREES

start_time = time()

# Create a extra trees regressor with default parameters
extra_trees_default = ExtraTreesRegressor()

# Compute the cross-validated negative root mean squared error
results['Extra trees'] = np.average(
                             cross_val_score(
                                 extra_trees_default,
                                 X_train,
                                 y_train,
                                 cv=cv,
                                 scoring='neg_root_mean_squared_error'
                                 )
                             )

# Record the execution time
exec_times['Extra trees'] = time() - start_time

In [ ]:
# HIST GRADIENT BOOSTING

start_time = time()

# Create a hist gradient boosting regressor with default parameters
hist_gradient_boosting_default = HistGradientBoostingRegressor()

# Compute the cross-validated negative root mean squared error
results['Hist gradient boosting'] = np.average(
                                        cross_val_score(
                                            hist_gradient_boosting_default,
                                            X_train,
                                            y_train,
                                            cv=cv,
                                            scoring='neg_root_mean_squared_error'
                                            )
                                        )

# Record the execution time
exec_times['Hist gradient boosting'] = time() - start_time

### 2. Grid Search for hyper-parameter tuning of KNN Regressor

In [ ]:
# K NEAREST NEIGHBOURS with GRID SEARCH

start_time = time()

# Define the hyperparameter grid for the KNN model
param_grid_knn = {
    'n_neighbors': np.arange(1, 10),
    'weights': ('uniform', 'distance'),
    'p': np.arange(1, 5, 1)
}

# Define the GridSearchCV object with the KNN regressor model, the hyperparameter grid, and the scoring method
k_neighbors_grid = GridSearchCV(
    KNeighborsRegressor(), 
    param_grid_knn,
    scoring='neg_root_mean_squared_error',
    cv=cv
)

# Fit the GridSearchCV object to the training data
k_neighbors_grid.fit(X=X_train, y=y_train)

# Save the best score and execution time
results['KNN + grid search'] = k_neighbors_grid.best_score_
exec_times['KNN + grid search'] = time() - start_time

### 3. Random Search for hyper-parameter tuning of Decision Tree Regressor.

In [ ]:
# DECISION TREE REGRESSOR with RANDOM SEARCH

start_time = time()

# Define the hyperparameter grid for random search
param_grid_tree = {
    'max_depth': list(range(2, 16)) + [None],
    'min_samples_split': np.arange(2, 16),
    'min_samples_leaf': np.arange(1, 16),
    'splitter': ('best', 'random'),
    'criterion': ('mse', 'friedman_mse', 'mae', 'poisson'),
    'max_features': ('auto', 'sqrt', 'log2')
}

# Set the budget
budget = 20

# Use RandomizedSearchCV to find the best hyperparameters
tree_regressor_rsearch = RandomizedSearchCV(
    estimator=DecisionTreeRegressor(), 
    param_distributions=param_grid_tree,
    scoring='neg_root_mean_squared_error',
    cv=cv,
    n_iter=budget
)

# Fit the randomized search object to the training data
tree_regressor_rsearch.fit(X=X_train, y=y_train)

# Save the best score and execution time
results['Decision tree regressor + random-search'] = tree_regressor_rsearch.best_score_
exec_times['Decision tree regressor + random-search'] = time() - start_time

### 4. Scikit-Optimize for hyper-parameter tuning of Extra Trees Regressor

In [ ]:
# EXTRA TREES with SCIKIT-OPTIMIZE

start_time = time()

# Define the search space for the hyperparameters
param_grid_etree = {
    'n_estimators': Integer(10, 1000),
    'max_depth': Integer(2, 16),
    'min_samples_split': Integer(2, 16),
    'min_samples_leaf': Integer(1, 16),
    'max_features': Categorical(('auto', 'sqrt', 'log2'))
}

# Set the budget
budget = 15

# Use BayesSearchCV to find the best hyperparameters
extra_trees_skopt = BayesSearchCV(
    ExtraTreesRegressor(),
    param_grid_etree,
    scoring='neg_root_mean_squared_error',
    cv=cv,
    n_iter=budget
)

# Fit the bayesian search object to the training data
extra_trees_skopt.fit(X=X_train, y=y_train)

# Save the best score and execution time
results['Extra trees + scikit-optimize'] = extra_trees_skopt.best_score_
exec_times['Extra trees + scikit-optimize'] = time() - start_time

### 5. Optuna for hyper-parameter tuning of HistGradientBoosting Regressor

In [ ]:
# Set the verbosity level of Optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Define a random sampler with a fixed seed for reproducibility
sampler = optuna.samplers.RandomSampler(seed=100469317)

In [ ]:
# HIST GRADIENT OBJECTIVE with OPTUNA

def hist_gradient_objective(trial):
    """
    Objective function for optimizing hyperparameters of
    HistGradientBoostingRegressor using Optuna.
    """
    loss = trial.suggest_categorical('loss', ('least_squares', 'least_absolute_deviation', 'poisson'))
    learning_rate = trial.suggest_float('learning_rate', 0.01, 1, step=0.01)
    max_iter = trial.suggest_int('max_iter', 10, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 16)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 8)

    # Create a Hist Gradient Boosting regressor with the suggested hyperparameters
    hist_gradient_boosting_tune = HistGradientBoostingRegressor(
        loss=loss,
        learning_rate=learning_rate,
        max_iter=max_iter,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf
    )

    # Calculate the average cross validation score using the regressor
    return np.average(cross_val_score(
        hist_gradient_boosting_tune,
        X_train,
        y_train,
        scoring='neg_root_mean_squared_error',
        n_jobs=1,
        cv=cv
    ))

start_time = time()

# Create an Optuna study for the Hist Gradient Boosting optimization
study_hist_gradient = optuna.create_study(direction="maximize", sampler=sampler)

# Optimize the study using the objective function and 25 trials
study_hist_gradient.optimize(hist_gradient_objective, n_trials=25)

# Store the best score in the results dictionary
results['Hist gradient boosting + Optuna'] = study_hist_gradient.best_value

In [ ]:
# Show the best score
results['Hist gradient boosting + Optuna']

-0.14656676356779083

### 6. Visualising of Optimization Process
Optuna's vizualization module allows for the plotting of the process of optimization that is taking place when optuna is used for hyper-parameter tuning.

In [17]:
# Hyperparameter importances plot using Optuna visualization
plot_hyperparameter_importances = optuna.visualization.plot_param_importances(study_hist_gradient)
plot_hyperparameter_importances.show()

This plot, called the hyper-parameter importances plot, is even more intuitive. It shows the magnitude of impact that a change in a given hyper-parameter will have on the objective value compared to the other hyper-parameters. As we can see for the plot, the hyper-parameter "learning_rate" is much, much more important than any other hyper-parameter we are tuning. It also shows that "loss" and all other parameters have essentially no impact whatsoever on the objective value.

In [18]:
# Parameter contour plot using Optuna visualization
plot_contour = optuna.visualization.plot_contour(study_hist_gradient, params=["loss","learning_rate"])
plot_contour.show()

This countour plot looking at the most and least important hyper-parameters clearly illustrates what was shown in the hyper-parameter importances plot. Taking any fixed value of "learning_rate" there is practically zero change in the shade of blue (which determines the objective value) for the three different values of "loss". The opposite is true when looking at the vice versa. Clearly, regardless of the value of "loss", as the value of "learning_rate" approaches 1, the objective value worsens.

Below, Optuna is used for hyper-parameter tuning as before but this time with pruning of unpromising trials.

In [19]:
# HIST GRADIENT BOOSTING with OPTUNA PRUNING

def objective_hist_gradient_boosting_steps(trial):
    # Suggest different hyperparameters to optimize
    loss = trial.suggest_categorical('loss', ('least_squares', 'least_absolute_deviation', 'poisson'))
    learning_rate = trial.suggest_float('learning_rate', 0.01, 1, step=0.01)
    max_iter = trial.suggest_int('max_iter', 10, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 16)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 8)
    
    # Create a Hist Gradient Boosting regressor with the suggested hyperparameters
    hist_gradient_boosting_tune = HistGradientBoostingRegressor(
        loss=loss,
        learning_rate=learning_rate,
        max_iter=max_iter,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf
    )
    
    # Evaluate the inner score using cross-validation
    inner_mse = np.average(cross_val_score(
        hist_gradient_boosting_tune,
        X_train,
        y_train, 
        scoring='neg_root_mean_squared_error',
        n_jobs=1,
        cv=cv
    ))
    
    # Report the intermediate score and prune trials that do not meet the pruning condition
    for step in range(100):
        trial.report(inner_mse, step)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return inner_mse


# Create a new Optuna study and optimize the hyperparameters
start_time = time()
study_PUT = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(),
    direction="maximize",
    sampler=sampler
)
study_PUT.optimize(objective_hist_gradient_boosting_steps, n_trials=25)

# Save the best score and execution time
results['Hist gradient boosting + Optuna pruning'] = study_PUT.best_value
exec_times['Hist gradient boosting + Optuna pruning'] = time() - start_time

### 7. Scikit-Optimize for hyper-parameter tuning of HistGradientBoosting Regressor

In [20]:
# HIST GRADIENT BOOSTING with SCIKIT-OPTIMIZE

start_time = time()

# Define the search space for hyperparameters
param_grid_histgradient = {
    'loss': ('least_squares', 'least_absolute_deviation', 'poisson'),
    'learning_rate': Real(0.01, 1),
    'max_iter': Integer(10, 1000),
    'max_depth': Integer(2, 16),
    'min_samples_leaf': Integer(2, 8)
}

# Set the budget
budget = 15

# Use BayesSearchCV to find the best hyperparameters
hist_gradient_boosting_skopt = BayesSearchCV(
    HistGradientBoostingRegressor(),
    param_grid_histgradient,
    scoring='neg_root_mean_squared_error',
    cv=cv,
    n_iter=budget
)

# Fit the bayesian search object to the training data
hist_gradient_boosting_skopt.fit(X=X_train, y=y_train)

# Save the best score and execution time
results['Hist gradient boosting + scikit-optimize'] = hist_gradient_boosting_skopt.best_score_
exec_times['Hist gradient boosting + scikit-optimize'] = time() - start_time

### 8. Naive Regressor for Comparison

In [21]:
# NAIVE REGRESSOR

start_time = time()

# Create a naive regressor that always predicts the mean of the target variable
naive_regressor = DummyRegressor()

# Compute the cross-validated negative root mean squared error
results['Naive regressor'] = np.average(
                                 cross_val_score(
                                     naive_regressor,
                                     X_train,
                                     y_train,
                                     cv=cv,
                                     scoring='neg_root_mean_squared_error'
                                     )
                                 )

# Record the execution time
exec_times['Naive regressor'] = time() - start_time

### 9. Performance Evaluation

In [22]:
# Loop through the results dictionary and print out the method name and score
for method, score in results.items():
    print(f'{method:.<44}{score:.8f}')

Decision tree regressor.....................-0.24690838
KNN.........................................-0.24687438
Random forest...............................-0.16309045
Extra trees.................................-0.15647485
Hist gradient boosting......................-0.15667534
KNN + grid search...........................-0.21798119
Decision tree regressor + random-search.....-0.20447928
Extra trees + scikit-optimize...............-0.15741547
Hist gradient boosting + Optuna.............-0.14656676
Hist gradient boosting + Optuna pruning.....-0.15162482
Hist gradient boosting + scikit-optimize....-0.14893854
Naive regressor.............................-0.40146892


In [23]:
# Loop through the execution times dictionary and print out the method name and execution time
for method, exec_time in exec_times.items():
    print(f'{method:.<44}{exec_time:.3f} s')

Decision tree regressor.....................0.030 s
KNN.........................................0.020 s
Random forest...............................1.767 s
Extra trees.................................1.495 s
Hist gradient boosting......................3.025 s
KNN + grid search...........................24.715 s
Decision tree regressor + random-search.....0.561 s
Extra trees + scikit-optimize...............32.561 s
Hist gradient boosting + Optuna pruning.....352.405 s
Hist gradient boosting + scikit-optimize....332.753 s
Naive regressor.............................0.003 s


Several results can be drawn from these tables. Firstly, all models perform better than the naive regressor, as expected. For KNN and the decision tree regressor the hyperparameter tuning has resulted in better scores. This is also true for all other models, but to a much lesser degree. The small improvement with different hyperparameter tuning methods for hist gradient boosting suggests that the small improvements are caused by the defaults values already being close to the optimum.

The execution times show how, as expected, extra trees are faster than random forest, albeit by a small margin. The best model is hist gradient boosting hyperparameter tuned with Optuna, even though scikit-optimize gets very close to its performance. It is hard to determine whether performance scales with time spent with hyperparameter tuning. The obvious response would be that it generally does, since larger budgets and wider ranges would be expected to increase performance at the cost of execution time. However, the results show that the biggest determinant on execution time is the model being tuned, with hist gradient boosting being particularly taxing.

### 10. Hyperparameter Tuning Extra-Trees and HistGradientBoosting with Optuna as a CASH Problem

In [24]:
def extra_trees_hist_gradient_cash_objective(trial):
    # Suggest a method to use from a list of two methods
    method = trial.suggest_categorical('method', ['hist_gradient_boosting', 'extra-trees'])

    # Configure the model according to the chosen method
    if method == 'hist_gradient_boosting':
        learning_rate_hist = trial.suggest_float("learning_rate", 0.01, 1, step=0.01)
        max_depth_hist = trial.suggest_int("max_depth", 2, 16)
        min_samples_leaf_hist = trial.suggest_int("min_samples_leaf", 2, 16)
        CASHclf = HistGradientBoostingRegressor(
                      learning_rate=learning_rate_hist,
                      max_depth=max_depth_hist,
                      min_samples_leaf=min_samples_leaf_hist
                      )
    else:
        max_depth_etrees = trial.suggest_int('max_depth2', 2, 16)
        min_samples_split_etrees = trial.suggest_int('min_samples_split', 2, 16)
        CASHclf = ExtraTreesRegressor(
                      max_depth=max_depth_etrees,
                      min_samples_split=min_samples_split_etrees
                      )

    # Evaluate the model using cross-validation
    return np.average(
               cross_val_score(
                   CASHclf,
                   X_train,
                   y_train, 
                   scoring='neg_root_mean_squared_error',
                   n_jobs=1,
                   cv=cv
                   )
               )

# Create an Optuna study to optimize the hyperparameters
study_cash = optuna.create_study(direction="maximize", sampler=sampler)
study_cash.optimize(extra_trees_hist_gradient_cash_objective, n_trials=20)

# Print the best hyperparameters and corresponding negative RMSE score
print(f'Best parameters: {study_cash.best_params}')
print(f'Negative root mean squared error: {study_cash.best_value}')

Best parameters: {'method': 'hist_gradient_boosting', 'learning_rate': 0.2, 'max_depth': 3, 'min_samples_leaf': 15}
Negative root mean squared error: -0.15063272002386519


Looking at the results, the absolute values for 'best value' are ~0.124 and ~0.293 when using extra-trees and histgradientboosting, respectively, with tuned hyper-parameters. Thus, we would expect the CASH problem to resort to the extra-trees method and perform some hyper-parameter tuning that would return a 'best value' close to 0.124. We got ~0.118 which tells us that the results are indeed consistent with what was obtained before.

### 11. Result with Best Performing Model on Test Data

In [25]:
# Initialize the HistGradientBoostingRegressor with the best parameters
hist_gradient_optuna_params = HistGradientBoostingRegressor(**study_hist_gradient.best_params)

# Fit the model on the training data
hist_gradient_optuna_params.fit(X_train, y_train)

# Make predictions on the test data
y_test_pred = hist_gradient_optuna_params.predict(X_test)

# Calculate the negative root mean squared error and print it
neg_rmse = -np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f'Negative mean squared error: {neg_rmse}')

Negative mean squared error: -0.11908161713672143


### 12. Halving Grid Search and Halving Random Search for Hyper-parameter Tuning with KNN and Decision Tree Regressor

What halving essentially means is that the hyper-parameter optimization process starts whilst using very few resources. As promising trials are identified, more resources are dedicated to the whereas unpromising trials are pruned. With HalvingGridSearch a tentative judgement on the performance of all candidate models is made before iteratively selecting and dedicating more resources to the candidates evaluated as having better performance. The difference with HalvingRandomSearch is that the judgement on performance is made on random candidates.

In [61]:
# K NEAREST NEIGHBOURS with HALVING GRID SEARCH

# Define the KNN regressor model with halving grid search and fit to the training data
k_neighbors_halving_grid = HalvingGridSearchCV(
    estimator=KNeighborsRegressor(),
    param_grid=param_grid_knn,
    scoring='neg_root_mean_squared_error',
    cv=cv
).fit(X=X_train, y=y_train)

# Save the best score
results['KNN + halving grid search'] = k_neighbors_halving_grid.best_score_

In [75]:
# DECISION TREE REGRESSOR with HALVING RANDOM SEARCH

# Define the decision tree regressor model with halving random search and fit to the training data
tree_regressor_halving_rsearch = HalvingRandomSearchCV(
    estimator=DecisionTreeRegressor(),
    param_distributions=param_grid_tree,
    scoring='neg_root_mean_squared_error',
    cv=cv
).fit(X_train, y_train)

# Save the best score
results['Decision tree regressor + halving random-search'] = tree_regressor_halving_rsearch.best_score_

In [76]:
# Loop through the results dictionary and print out the method name and score
for method, score in results.items():
    print(f'{method:.<51}{score:.8f}')

Decision tree regressor............................-0.24690838
KNN................................................-0.24687438
Random forest......................................-0.16309045
Extra trees........................................-0.15647485
Hist gradient boosting.............................-0.15667534
KNN + grid search..................................-0.21798119
Decision tree regressor + random-search............-0.20447928
Extra trees + scikit-optimize......................-0.15741547
Hist gradient boosting + Optuna....................-0.14656676
Hist gradient boosting + Optuna pruning............-0.15162482
Hist gradient boosting + scikit-optimize...........-0.14893854
Naive regressor....................................-0.40146892
KNN + halving grid search..........................-0.21542997
Decision tree regressor + halving random-search....-0.21423049
